## Inicialización

## Modelado

Define las funciones necesarias para entrenar tu modelo en la plataforma GPU y crear un solo script que las contenga todas junto con la sección de inicialización.

Para facilitar esta tarea, puedes definirlas en este notebook y ejecutar un código listo en la siguiente sección para componer automáticamente el script.

Los revisores del proyecto también verificarán las definiciones a continuación, para que puedan comprender cómo construiste el modelo.

In [11]:
# Función para cargar los datos de entrenamiento para el modelo
def load_train(path):
    """
    Carga la parte de entrenamiento del conjunto de datos desde la ruta.
    """
    labels_path = path + 'labels.csv'
    labels_df = pd.read_csv(labels_path)

    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.25
    )

    train_gen_flow = datagen.flow_from_dataframe(
        dataframe=labels_df,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='training',
        seed=12345
    )

    return train_gen_flow

In [12]:
# Función para cargar los datos de prueba para el modelo
def load_test(path):
    """
    Carga la parte de validación/prueba del conjunto de datos desde la ruta.
    """
    labels_path = path + 'labels.csv'
    labels_df = pd.read_csv(labels_path)

    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.25
    )

    test_gen_flow = datagen.flow_from_dataframe(
        dataframe=labels_df,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='validation',
        seed=12345
    )

    return test_gen_flow

In [13]:
# Función para crear el modelo de entrenamiento
def create_model(input_shape):
    """
    Define el modelo.
    """
    model = Sequential()
    model.add(ResNet50(weights='imagenet', include_top=False, input_shape=input_shape))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))  # La salida es la edad, es decir un solo valor

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mae'])

    return model

In [14]:
# Función para entrenar el modelo
def train_model(model, train_data, test_data, batch_size=None, epochs=5, steps_per_epoch=None, validation_steps=None):
    """
    Entrena el modelo dados los parámetros.
    """
    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    model.fit(
        train_data,
        steps_per_epoch=steps_per_epoch,
        validation_data=test_data,
        validation_steps=validation_steps,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    return model

### Prepara el script para ejecutarlo en la plataforma GPU

Una vez que hayas definido las funciones necesarias, puedes redactar un script para la plataforma GPU, descargarlo a través del menú "File|Open..." (Archivo|Abrir) y cargarlo más tarde para ejecutarlo en la plataforma GPU.

Nota: el script debe incluir también la sección de inicialización. A continuación se muestra un ejemplo.

In [15]:
# prepara un script para ejecutarlo en la plataforma GPU

init_str = """
import pandas as pd

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
"""

import inspect

with open('run_model_on_gpu.py', 'w') as f:
    
    f.write(init_str)
    f.write('\n\n')
        
    for fn_name in [load_train, load_test, create_model, train_model]:
        
        src = inspect.getsource(fn_name)
        f.write(src)
        f.write('\n\n')

### Resultado

Coloca aquí el resultado de la plataforma GPU como una celda Markdown.

El resultado al correr el código localmente en el GPU es:

c:\Users\ricar\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()  
  
  
**Epoch 1/5**
190/190 ━━━━━━━━━━━━━━━━━━━━ 1126s 6s/step - loss: 305.3067 - mae: 12.7807 - val_loss: 668.8609 - val_mae: 20.7864

**Epoch 2/5**
190/190 ━━━━━━━━━━━━━━━━━━━━ 1069s 6s/step - loss: 67.3475 - mae: 6.3674 - val_loss: 420.6157 - val_mae: 15.3095

**Epoch 3/5**
190/190 ━━━━━━━━━━━━━━━━━━━━ 1079s 6s/step - loss: 44.0825 - mae: 5.0434 - val_loss: 267.6707 - val_mae: 12.2935

**Epoch 4/5**
190/190 ━━━━━━━━━━━━━━━━━━━━ 1065s 6s/step - loss: 35.0197 - mae: 4.3925 - val_loss: 132.2638 - val_mae: 9.0930

**Epoch 5/5**
190/190 ━━━━━━━━━━━━━━━━━━━━ 1045s 6s/step - loss: 29.7083 - mae: 4.1177 - val_loss: 79.1488 - val_mae: 6.8434


## Conclusiones

Se usó una **red neuronal implementando ResNet50 para procesamiento de imágenes en el proyecto de regresión lineal**. 
El **modelo fue capaz de identificar correctamente la edad de las personas por medio del conjunto de 7651 imágenes** y una semilla aleatorizada de 12345.
El **desempeño del modelo es satisfactorio**, con un pequeño margen para posible mejora, implementando ajuste de hiperparámetros o herramientas como GridSearchCV.

El **equipo utilizado es un GPU(Graphics Processing Unit) Nvidia GTX Geforce 1650**. 

- Debido a los recursos limitados en la plataforma, la terminal moría durante el entrenamiento, por lo que el modelo se entrenó localmente  con un conjunto de 7651 imágenes y mostró una **progresión positiva en el entranamiento para los conjuntos de entrenamiento y validación**, reduciendo los valores de pérdida en cada época
- Demoró 89 minutos ajustado para 5 épocas. 
- **No se rebasó el umbral máximo de margen de error en el EAM (Error Absoluto Medio) de 8**
- **El mejor valor de EAM (Error Absoluto Medio) fue en la última época con 4.11** 
- El valor de **pérdida general en el conjunto de datos es de 29.70** 
- un valor de **pérdida de 79.14 en el conjunto de validación**
- **EAM de 6.84 en el conjunto de validación**